# Final_term
17011724 최시우 / 17011730 이유진 / 17011739 이재훈

In [1]:
import pandas as pd
import numpy as np
import jester_data

In [2]:
rate=jester_data.m
rate

array([[4.08, -0.29, 6.36, ..., 0.34, -4.32, 1.07],
       [-6.17, -3.54, 0.44, ..., -9.08, -5.05, -3.45],
       [6.84, 3.16, 9.17, ..., -0.05, 1.31, 0.0],
       ...,
       [5.34, -9.71, 4.95, ..., -1.12, -5.63, 5.68],
       [7.67, 8.2, 0.78, ..., -2.23, -7.91, -7.52],
       [-7.57, -4.71, -9.32, ..., -9.03, -4.37, -4.81]], dtype=object)

In [3]:
rate=(rate+10)/20 # 0-1정규화
rate10=rate[:10,:]

# 1) Singularity

In [19]:
Sp=np.zeros([100]) ; Sn=np.zeros([100])
Sp=1-(sum(rate>0.5)/1000)
#Sn=1-(sum(rate<=0.5)/1000)
Sn=1-Sp  #Sp+Sn=1

In [18]:
def singularity(a):
    Sim=np.zeros([1000,1000])
    for u in range(1000):
        for v in range(u,1000):
            AA=np.zeros([100]) ; BB=np.zeros([100]) ; CC=np.zeros([100])
            U=10
            
            #case1
            #A=len(np.where((a[u,:]>0.5) & (a[v,:]>0.5))[0])
            index=np.where((a[u,:]>0.5) & (a[v,:]>0.5))
            A=len(index[0])
            AA[index]=AA[index]+(1-(a[u,index]-a[v,index])**2)*(Sp[index]**2)
            
            
            #case2
            #B=len(np.where((a[u,:]<=0.5) & (a[v,:]<=0.5))[0])
            index=np.where((a[u,:]<=0.5) & (a[v,:]<=0.5))
            B=len(index[0])
            BB[index]=BB[index]+(1-(a[u,index]-a[v,index])**2)*(Sn[index]**2)
                     
                
            #case3
            #C=len(np.where(((a[u,:]>0.5) & (a[v,:]<=0.5)) | ((a[u,:]<=0.5) & (a[v,:]>0.5)))[0])
            index=np.where(((a[u,:]>0.5) & (a[v,:]<=0.5)) | ((a[u,:]<=0.5) & (a[v,:]>0.5)))
            C=len(index[0])
            CC[index]=CC[index]+(1-(a[u,index]-a[v,index])**2)*(Sp[index]*Sn[index])
           
            if A==0 or B==0 or C==0:
                Sim[u,v]=0
                Sim[v,u]=Sim[u,v]
            else:
                Sim[u,v]=(sum(AA)/A + sum(BB)/B + sum(CC)/C)/3
                Sim[v,u]=Sim[u,v]
                
    return Sim

In [63]:
result_s=singularity(rate)
result_s

array([[0.        , 0.2509152 , 0.23148972, ..., 0.19933016, 0.18067581,
        0.19576144],
       [0.2509152 , 0.        , 0.23804159, ..., 0.19834897, 0.21139385,
        0.20270659],
       [0.23148972, 0.23804159, 0.        , ..., 0.18973412, 0.19246311,
        0.18624222],
       ...,
       [0.19933016, 0.19834897, 0.18973412, ..., 0.        , 0.16744105,
        0.15926692],
       [0.18067581, 0.21139385, 0.19246311, ..., 0.16744105, 0.        ,
        0.16772257],
       [0.19576144, 0.20270659, 0.18624222, ..., 0.15926692, 0.16772257,
        0.        ]])

# 2) PIP

### Proximity x Impact x Popularity

## - avg

In [5]:
avg=[] #아이템 평점의 평균을 구하는 시간을 줄이기 새로운 리스트를 할당하여 미리 구해놓은 뒤 활용
for i in range (100):
    avg.append(np.mean(rate[:,i]))

## - Rmax, Rmin, Rm

In [6]:
Rmax=1.0
Rmin=0.0
Rm=(Rmax+Rmin)/2

## - Agreement

In [7]:
def Agreement(a,b):
    agreement=np.array(a*b>=Rm,dtype=np.int)
    agreement=np.where(agreement==0,2,agreement)
    #print(agreement)
    return agreement

## - Proximity 

In [8]:
def Proximity(a,u,v):
    proximity=np.zeros([100])
    c=Agreement(a[u,:], a[v,:]) #True->1  /  False->2
    proximity=((2*(Rmax-Rmin)+1)-(c*np.abs(a[u,:]-a[v,:])))**2 
    return proximity

## - Impact

In [9]:
def Impact(a,u,v):
    impact=np.zeros([100])
    agreement=Agreement(a[u,:],a[v,:])
    
    impact=np.where(agreement==1,(np.abs(a[u,:]-Rm)+1)*(np.abs(a[v,:]-Rm)+1),impact)
    impact=np.where(agreement==2,1/((np.abs(a[u,:]-Rm)+1)*(np.abs(a[v,:]-Rm)+1)),impact)
    #print(impact)
    return impact

## - Popularity

In [10]:
def Popularity(a,u,v):
    popularity=np.zeros([100])
    popularity=np.where(((a[u,:]>avg) & (a[v,:]>avg)) | ((a[u,:]<avg) & (a[v,:]<avg)),1+((a[u,:]+a[v,:])/2-avg)**2,popularity)
    popularity=np.where(((a[u,:]>avg) & (a[v,:]<avg)) | ((a[u,:]<avg) & (a[v,:]>avg)),1,popularity)
    return popularity

## - PIP

In [11]:
def PIP(a):
    n=len(a)
    PIP=np.zeros([n,n])
    for u in range(n):
        for v in range(u, n):
            tmp=0.0
            tmp = Proximity(a,u,v) * Impact(a,u,v) * Popularity(a,u,v)
            
            PIP[u,v]=sum(tmp)
            PIP[v,u]=PIP[u,v]
    return PIP

In [77]:
result_p=PIP(rate)
result_p

array([[1268.05274231,  379.85163506,  485.70681026, ...,  550.58087281,
         593.10566971,  419.60511266],
       [ 379.85163506,  658.47433055,  490.90917642, ...,  389.96504972,
         364.13820195,  484.1146717 ],
       [ 485.70681026,  490.90917642,  818.66136811, ...,  554.61962631,
         534.825774  ,  476.12827633],
       ...,
       [ 550.58087281,  389.96504972,  554.61962631, ...,  955.61596031,
         654.19053648,  437.89497041],
       [ 593.10566971,  364.13820195,  534.825774  , ...,  654.19053648,
        1115.73060067,  493.08880442],
       [ 419.60511266,  484.1146717 ,  476.12827633, ...,  437.89497041,
         493.08880442,  705.27938582]])

# 2) Verification

singularity 와 pip의 similarity로 predicted rating을 구한 후 비교

In [79]:
mean=np.mean(rate,axis=1)

In [67]:
len(mean)

1000

In [15]:
def predicted(mat,sim,k):
    predicted_rating=np.zeros([1000,100])
    mean=np.nanmean(np.where(mat!=0, mat, np.nan),axis=1)#user mean
        
    k_neighbors = np.argsort(-sim)
    k_neighbors = np.delete(k_neighbors, np.s_[k:], 1)
    
    NumUsers=np.size(mat,axis=0) #10
    
    for u in range(0,NumUsers):
        list_sim=sim[u,k_neighbors[u,]]
        list_rating=mat[k_neighbors[u, ], ].astype('float64')
        list_mean=mean[k_neighbors[u, ], ]
        
        denominator=np.sum(list_sim)
        if denominator==0:
            predicted_rating[u]=0
        else:
            numerator=np.sum(list_sim.reshape(-1,1)*(list_rating - list_mean.reshape(-1,1)),axis=0)
            predicted_rating[u] = mean[u] + (numerator/ denominator)
     
        
    return predicted_rating

### - PIP

In [13]:
sim_p=PIP(rate)

In [16]:
pred_p=predicted(rate,sim_p,1000)
pred_p

array([[0.64766148, 0.62284418, 0.61636679, ..., 0.60909845, 0.5989955 ,
        0.6529502 ],
       [0.2691318 , 0.24225655, 0.22798816, ..., 0.22516915, 0.21242665,
        0.26865072],
       [0.39745487, 0.37207399, 0.36000818, ..., 0.35174644, 0.33855483,
        0.39451082],
       ...,
       [0.44381303, 0.42148807, 0.40938234, ..., 0.40814893, 0.39029882,
        0.45262561],
       [0.55549844, 0.53378594, 0.51734745, ..., 0.51343968, 0.50110228,
        0.55530916],
       [0.32741204, 0.30333084, 0.28789353, ..., 0.28778876, 0.27573969,
        0.32946952]])

### - singularity

In [22]:
sim_s=singularity(rate)

In [23]:
pred_s=predicted(rate,sim_s,1000)
pred_s

array([[0.65021099, 0.62411338, 0.6115894 , ..., 0.60706796, 0.59765596,
        0.6511318 ],
       [0.26929062, 0.24050752, 0.23265949, ..., 0.22193147, 0.21413495,
        0.26782601],
       [0.39567064, 0.36823428, 0.35660157, ..., 0.34906254, 0.33906646,
        0.3932642 ],
       ...,
       [0.44665824, 0.42056133, 0.40836387, ..., 0.40395466, 0.39103598,
        0.44843969],
       [0.55652571, 0.53073445, 0.51755178, ..., 0.51140872, 0.50092962,
        0.55503946],
       [0.33047736, 0.3032941 , 0.29064171, ..., 0.28634865, 0.27640107,
        0.32867905]])

### - pip  error

In [24]:
error_pip=(rate-pred_p).astype('float')
error_pip.round(3)

array([[ 0.056, -0.137,  0.202, ..., -0.092, -0.315, -0.099],
       [-0.078,  0.081,  0.294, ..., -0.179,  0.035,  0.059],
       [ 0.445,  0.286,  0.598, ...,  0.146,  0.227,  0.105],
       ...,
       [ 0.323, -0.407,  0.338, ...,  0.036, -0.172,  0.331],
       [ 0.328,  0.376,  0.022, ..., -0.125, -0.397, -0.431],
       [-0.206, -0.039, -0.254, ..., -0.239,  0.006, -0.07 ]])

### - singularity error

In [25]:
error_sing=(rate-pred_s).astype('float')
error_sing.round(3)

array([[ 0.054, -0.139,  0.206, ..., -0.09 , -0.314, -0.098],
       [-0.078,  0.082,  0.289, ..., -0.176,  0.033,  0.06 ],
       [ 0.446,  0.29 ,  0.602, ...,  0.148,  0.226,  0.107],
       ...,
       [ 0.32 , -0.406,  0.339, ...,  0.04 , -0.173,  0.336],
       [ 0.327,  0.379,  0.021, ..., -0.123, -0.396, -0.431],
       [-0.209, -0.039, -0.257, ..., -0.238,  0.005, -0.069]])

## Compare

In [26]:
ep=np.mean(error_pip.round(3))
es=np.mean(error_sing.round(3))
print('error_mean_PIP = ',ep,'\nerror_mean_singulrity = ',es)

error_mean_PIP =  -7.500000000001095e-07 
error_mean_singulrity =  0.0019029300000000002
